In [ ]:
# Imports
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir
import numpy as np

import pandas as pd
from aequilibrae.project.database_connection import database_connection
from aequilibrae.utils.db_utils import read_and_close

from aequilibrae.transit import Transit
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae.utils.create_example import create_example
from aequilibrae.matrix import AequilibraeMatrix


# TODO:
# 1. Figure out the capacity vector and ordering etc.
# 2. Update reference files so tests work
# 3. Build PC, understand how venv's and wsl system works from ground up, ubuntu 22.04, 
# 4. Remove this notebook from git.

# Rewrite TODO's!

In [ ]:
# Build temporary updated coquimbo file

fldr = join(gettempdir(), uuid4().hex)
print(fldr)
project = create_example(fldr, "coquimbo")

remove(join(fldr, "public_transport.sqlite"))
dest_path = join(fldr, "gtfs_coquimbo.zip")

data = Transit(project)

transit = data.new_gtfs_builder(agency="Lisanco", file_path=dest_path)

transit.load_date("2016-04-13")

# Now we execute the map matching to find the real paths.
# Depending on the GTFS size, this process can be really time-consuming.
transit.set_allow_map_match(True)
transit.map_match()

# Finally, we save our GTFS into our model.
transit.save_to_disk()

In [ ]:
# Attempt to build preload with new pce column

project.network.build_graphs()
project.activate()

g = project.network.graphs["c"]
g.set_skimming(["travel_time"])
g.set_graph("travel_time")
g.set_blocked_centroid_flows(False)
g.graph["capacity"] = 500
g.graph["travel_time"] = g.graph["distance"] / 50

def hr_to_sec(e):
    return int(e * 60 * 60)

start = hr_to_sec(7)
end = hr_to_sec(8)
inclusion_cond = "start"

In [ ]:
n_zones = g.centroids.shape[0]
matrix = AequilibraeMatrix()
matrix.create_empty(zones=n_zones, matrix_names=["car"])
matrix.index = g.centroids

matrix.matrices[:, :, 0] = 5  # 5 trips from each OD should cause a little bit of congestion
matrix.computational_view("car")

In [ ]:
assignment = TrafficAssignment()
assignment.set_classes([TrafficClass("car", g, matrix)])

assignment.set_vdf("BPR")
assignment.set_vdf_parameters({"alpha": 0.15, "beta": 4.0})
assignment.set_capacity_field("capacity")
assignment.set_time_field("travel_time")
assignment.max_iter = 1  # AON assignment
assignment.set_algorithm("msa")

In [ ]:
transit = Transit(project)
preload = transit.build_pt_preload(start, end, inclusion_cond=inclusion_cond)
preload

In [ ]:
assignment.add_preload(preload)
assignment.preloads

In [ ]:
# %%
matrix.close()
project.close()